In [4]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv
import pprint
import highway_env  # noqa: F401

TRAIN = True

n_cpu = 6
batch_size = 64
env = make_vec_env("racetrack-v0", n_envs=n_cpu, vec_env_cls=SubprocVecEnv)

## Train model and save it

In [7]:
model = PPO(
    "MlpPolicy",
    env,
    policy_kwargs=dict(net_arch=[dict(pi=[256, 256], vf=[256, 256])]),
    n_steps=batch_size * 12 // n_cpu,
    batch_size=batch_size,
    n_epochs=10,
    learning_rate=5e-4,
    gamma=0.9,
    verbose=2,
    tensorboard_log="racetrack_ppo/",
)
# Train the model
if TRAIN:
    model.learn(total_timesteps=int(1e2)) # try with 1e5 but takes a very long time!!
    model.save("racetrack_ppo/model")
    # del model

Using cuda device
Logging to racetrack_ppo/PPO_4
----------------------------
| time/              |     |
|    fps             | 80  |
|    iterations      | 1   |
|    time_elapsed    | 9   |
|    total_timesteps | 768 |
----------------------------


## Vizualise in a video

In [9]:
# Run the algorithm
model = PPO.load("racetrack_ppo/model", env=env)

env = gym.make("racetrack-v0")
env = RecordVideo(
    env, video_folder="racetrack_ppo/videos", episode_trigger=lambda e: True
)
env.unwrapped.set_record_video_wrapper(env)

for video in range(100):
    done = truncated = False
    obs, info = env.reset()
    while not (done or truncated):
        # Predict
        action, _states = model.predict(obs, deterministic=True)
        # Get reward
        obs, reward, done, truncated, info = env.step(action)
        # Render
        env.render()
env.close()

UnpicklingError: Weights only load failed. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution.Do it only if you get the file from a trusted source. WeightsUnpickler error: Unsupported operand 71